# RAG

# Initialisation

In [1]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
	# Commandes à exécuter uniquement sur Google Colab
	!git clone https://github.com/Zbrlaa/Rag
	%cd Rag
	!pip install -r requirements.txt
else:
	print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

fatal: destination path 'Rag' already exists and is not an empty directory.
/content/Rag


# Exercice 1 : Indexation

In [2]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("data/", use_multithreading=True)
documents = loader.load()
print(f"{len(documents)} documents")

65 documents


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
	chunk_size=1000,
	chunk_overlap=200,
	add_start_index=True,
	separators=["\n\n", "\n", " ", ""] #Ordre de priorité des caractères de découpe
)

all_splits = text_splitter.split_documents(documents)

print(f"Nombre de paragraphes : {len(all_splits)}")

Nombre de paragraphes : 8919


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
	model_name="intfloat/multilingual-e5-base"
)

/tmp/ipython-input-464981128.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
	documents=all_splits,
	embedding=embedding_model,
	persist_directory="./chroma_db"
)

print(f"Nombre d'éléments dans la base : {vectorstore._collection.count()}")

Nombre d'éléments dans la base : 8919


# Exercice 2 : Interrogation

In [6]:
def ask_base(query, k=3):
    # Recherche docs les plus proches et scores de distance
    docs_and_scores = vectorstore.similarity_search_with_score(query, k)

    return docs_and_scores

In [7]:
ma_query = "IA driving"

result = ask_base(ma_query, 2)

for doc, score in result:
	print(f"- Score de distance : {score:.4f}")
	print(f"- Source : {doc.metadata.get('source', 'Inconnue')}")
	print(f"- Start Index : {doc.metadata.get('start_index', 0)}")
	print("-" * 50)

- Score de distance : 0.3397
- Source : data/autres_articles/2412.18607v1.pdf
- Start Index : 8737
--------------------------------------------------
- Score de distance : 0.3594
- Source : data/autres_articles/2412.18607v1.pdf
- Start Index : 15032
--------------------------------------------------


# Exercice 3 : Prompt Template

In [8]:
from langchain_core.prompts import PromptTemplate

template = """Vous êtes un assistant spécialisé dans l'analyse de documents.
Utilisez les éléments de contexte suivants pour répondre à la question posée.
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE :
{context}

QUESTION :
{question}

RÉPONSE :"""

rag_prompt = PromptTemplate(
	template=template,
	input_variables=["context", "question"]
)

print(rag_prompt.format(context="1", question="2"))

Vous êtes un assistant spécialisé dans l'analyse de documents. 
Utilisez les éléments de contexte suivants pour répondre à la question posée. 
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE : 
1

QUESTION : 
2

RÉPONSE :


# Exercice 4 : chaîne RAG

In [10]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [12]:
def ask_llm_with_rag(question, k=10):
	docs_and_scores = ask_base(question, k=k)

	context = "\n\n".join([doc.page_content for doc, score in docs_and_scores])

	prompt_final = rag_prompt.format(context=context, question=question)

	response = llm.invoke(prompt_final)

	return response.content

ask_llm_with_rag("IA driving ?")

'D\'après les éléments de contexte fournis, la conduite assistée par IA (ou conduite autonome) est un domaine de recherche et de développement actif, caractérisé par diverses approches et modèles :\n\n1.  **Conduite autonome de bout en bout (End-to-end Autonomous Driving)** : C\'est un concept central, où le système apprend à conduire directement à partir des capteurs (vision, etc.) jusqu\'aux actions de conduite.\n    *   Des travaux comme "End to end learning for self-driving cars" [2] et "End-to-end autonomous driving: Challenges and frontiers" [10] en sont des exemples.\n    *   Des modèles spécifiques incluent "St-p3: End-to-end vision-based autonomous driving via spatial-temporal feature learning" [28] et "Neat: Neural attention fields for end-to-end autonomous driving" [12].\n\n2.  **Apprentissage par imitation (Imitation Learning)** : Une méthode courante où le système apprend à imiter le comportement de conducteurs humains.\n    *   "End-to-end driving via conditional imitatio

# Exercice 5 : mémoire